In [20]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import IBMQ
from qiskit import pulse            
from qiskit.pulse import pulse_lib 
from qiskit import assemble
from qiskit.tools.monitor import job_monitor

def init(backend_name):
    print(f'Initializing IBMQ {backend_name}')
    IBMQ.load_account()

    provider_anl = IBMQ.get_provider(hub='ibm-q-ornl', group='bes-qis', 
                                     project='argonne')
    provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
    backend = provider.get_backend(backend_name)
    return backend
    
def init_open_pulse(backend_name='ibmq_armonk'):
    backend = init(backend_name)
    backend_config = backend.configuration()
    assert backend_config.open_pulse, "Backend doesn't support OpenPulse"
    return backend

# samples need to be multiples of 16
def get_closest_multiple_of_16(num):
    return (int(num) - (int(num)%16))


In [21]:
from utils import (
    make_drive_pulse, make_meas_pulse
    ,freq_sweep_schedule
    ,sweep_program
    ,qbit_freqs
)

In [22]:
backend = init_open_pulse()

Initializing IBMQ ibmq_armonk


In [23]:
frequency_sweep_program = sweep_program(backend, frequencies=None)

Estimated frequency for qbit 0 is 4.974292853304637


NameError: name 'np' is not defined

In [ ]:
job = backend.run(frequency_sweep_program)
job_monitor(job)
job.error_message()

In [ ]:
def unpack_results(results):
    sweep_values = []
    for i in range(len(results.results)):
        # Get the results from the ith experiment
        res = results.get_memory(i)
        # Get the results for `qubit` from this experiment
        sweep_values.append(res[qbit])
    return sweep_values

results = job.result(timeout=120)

Job Status: job is queued (30)      

In [7]:
frequency_sweep_results = job.result(timeout=120) # timeout parameter set to 120 seconds
print(frequency_sweep_results)


plt.scatter(frequencies, sweep_values, color='black') # plot real part of sweep values
plt.xlim([min(frequencies), max(frequencies)])
plt.xlabel("Frequency [GHz]")
plt.ylabel("Measured signal [a.u.]")
plt.show()

### Fit results

from scipy.optimize import curve_fit

def fit_function(x_values, y_values, function, init_params):
    fitparams, conv = curve_fit(function, x_values, y_values, init_params)
    y_fit = function(x_values, *fitparams)
    
    return fitparams, y_fit

fit_params, y_fit = fit_function(frequencies,
                                 sweep_values, 
                                 lambda x, A, q_freq, B, C: (A / np.pi) * (B / ((x - q_freq)**2 + B**2)) + C,
                                 [-2e10, 4.975, 1, 3e10] # initial parameters for curve_fit
                                )
print(fit_params)

plt.scatter(frequencies, sweep_values, color='black')
plt.plot(frequencies, y_fit, color='red')
plt.xlim([min(frequencies), max(frequencies)])

plt.xlabel("Frequency [GHz]")
plt.ylabel("Measured Signal [a.u.]")
plt.show()

print("Fit center:",fit_params[1])
print("IBM center:", center_est_freq)
print("IBM error:", (fit_params[1]-center_est_freq)/fit_params[1])

NameError: name 'job' is not defined